In [18]:
import os
import re
import glob
from pathlib import Path
import shutil
import time
import numpy as np
import pandas as pd
from lxml import etree
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from progressbar import ProgressBar

In [2]:
# Configurations.
doi_url_pattern = "https://www.doi.org/{doi:s}"
# doi_url_pattern = "https://sci-hub.ru/{doi:s}"
download_path = os.path.join(os.getcwd(), "download")
save_path = os.path.join(os.getcwd(), "pdf")

# Load paper list requiring abstracts.
df = pd.read_csv("./progress.csv")
df = df.loc[:, ["id", "title", "doi", "pdf"]]
print(len(df))
df.head(2)

72


,id,title,doi,pdf
0,5,A Method to Evaluate Spectral Analysis by Spec...,10.3390/s22155638,NaN
1,6,A Miniaturized Flexible Functional Near-infrar...,10.1109/ISCAS48785.2022.9937752,NaN


In [81]:
# Start a webdriver instance.
service = Service(executable_path="./chromedriver.exe")
options = webdriver.ChromeOptions()
prefs = {}
prefs["profile.default_content_settings.popups"] = 0
prefs["download.default_directory"] = download_path
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(service=service, options=options)

In [4]:
def download_wait(directory, timeout=30, nfiles=None):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds to wait until timing out.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    seconds = 0
    dl_wait = True
    while dl_wait and seconds < timeout:
        time.sleep(1)
        dl_wait = False
        files = os.listdir(directory)
        if nfiles and len(files) != nfiles:
            dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'):
                dl_wait = True

        seconds += 1
    return seconds

In [5]:
# Init generator. 
df_iter = df.iterrows()

In [174]:
# Iterating through papers.
pbar = ProgressBar(max_value=len(df))

index, row = next(df_iter)
    
# Go to page.
try:
    doi = row.doi
    url = doi_url_pattern.format( doi=doi)
    # print(url, end=" ")
    print(f"{index}: {url}")
    driver.get(url)

#     button = driver.find_element_by_xpath("//div[@id='buttons']//button")
#     button.click()

    # Wait until downloaded. 
    # download_wait(download_path)

    # print(" pdf downloaded")

except Exception as e:
    print(index, end=":")
    print(doi, end=" ")
    print(str(e))

# Update progress.
pbar.update(index+1)

StopIteration: 

In [173]:
# Move PDF.
# re_pdf = re.compile(".*/(?P<pdf_name>.*?\.pdf).*")
pdf_files = glob.glob(download_path + "/*.pdf")
if len(pdf_files) == 1:
    # pdf_name = Path(pdf_files[0]).stem
    pdf_savepath = os.path.join(save_path, "{:04d}_{}.pdf".format(row.id, doi.replace("\\", "").replace("/", "__")))
    shutil.move(pdf_files[0], pdf_savepath)
    
    print(f"{pdf_files[0]} --> {pdf_savepath}")

    # Record.
    df.loc[index, "pdf"] = True
else:
    print(f"Error: {len(pdf_files)} pdf files found.")

D:\projects\Crawlers\Doi-Crawler\download\sensors-22-09704.pdf --> D:\projects\Crawlers\Doi-Crawler\pdf\0115_10.3390__s22249704.pdf


In [163]:
url

'https://www.doi.org/10.1016/j.compag.2022.106945'

In [26]:
row.id

5